In [1]:
:opt no-lint

# 25 Composing Types
## 25.1 Composing types
Functors and applicatives are both closed under composition: composing two functors (or two applicatives) results into another functor (or applicative). But composing two monads may not result into another monad.
However, composing monads may be desirable, as different monads allow us to work with different effects.
A monad transformer is a variant of an ordinary type that takes an additional type argument that is assumed to have a `Monad` instance. The transformer variant of a type gives us a `Monad` instance that binds over both bits of structure.
## 25.2 Common functions as types
Type constructors can take other type constructors as arguments, just as functions can take other functions as arguments. This is what allows us to compose types.

In [2]:
newtype Compose f g a = Compose { getCompose :: f (g a) } deriving (Eq, Show)

:t Compose [[1, 2, 3]]
:t Compose (Just [1, 2, 3])
:t Compose [Just 1, Nothing]

Compose [[1, 2, 3]] :: forall a. Num a => Compose [] [] a

Compose (Just [1, 2, 3]) :: forall a. Num a => Compose Maybe [] a

Compose [Just 1, Nothing] :: forall a. Num a => Compose [] Maybe a

## 25.3 Two functors sittin’ in a tree, L-I-F-T-I-N-G
The composition of two datatypesvthat have a `Functor` instance gives rise to a new `Functor` instance:

In [3]:
instance (Functor f, Functor g) => Functor (Compose f g) where
  fmap f (Compose fga) = Compose $ (fmap . fmap) f fga
  
(+1) <$> Compose [[1, 2, 3]]

Compose {getCompose = [[2,3,4]]}

In fact, `fmap . fmap` satisfies all the functor laws.
## 25.4 Twinplicative
Applicatives are also closed under composition.
#### GOTCHA! Exercise time

In [4]:
{-# LANGUAGE InstanceSigs #-}

instance (Applicative f, Applicative g) => Applicative (Compose f g) where
  pure :: a -> Compose f g a
  pure = Compose . pure . pure
  (<*>) :: Compose f g (a -> b) -> Compose f g a -> Compose f g b
  (Compose f) <*> (Compose a) = Compose $ fmap (<*>) f <*> a

Compose [[(+1)]] <*> Compose [[1, 2, 3]]

Compose {getCompose = [[2,3,4]]}

## 25.5 Twonad?
Within `Compose`, both types are polymorphic (e.g.: `[]` and `Maybe`). And it is not possible to combine two polymorphic binding into a single  `>>=` for `Monad`.
To combine the two types we need a _monad transformers_ instead.

A functor applies a computation to a value inside a structure, hence it is well defined ureagarding the complexity of the structure. An applicative combines independent computations. A monad, on the other side, combines dependent computations.

## 25.6 Exercises: Compose instances
1. Write the `Compose` `Foldable` instance.

In [5]:
instance (Foldable f, Foldable g) => Foldable (Compose f g) where
  foldMap f (Compose fga) = (foldMap . foldMap) f fga

2. Write the `Compose` `Traversable` instance:

In [6]:
instance (Traversable f, Traversable g) => Traversable (Compose f g) where
  traverse f (Compose fga) = Compose <$> (traverse . traverse) f fga

#### And now for something completely different

In [7]:
{-# LANGUAGE InstanceSigs #-}

class Bifunctor p where
  {-# MINIMAL bimap | first, second #-}
  bimap :: (a -> b) -> (c -> d) -> p a c -> p b d
  bimap f g = first f . second g

  first :: (a -> b) -> p a c -> p b c
  first f = bimap f id

  second :: (b -> c) -> p a b -> p a c
  second = bimap id

Write `Bifunctor` instances for the following types

1. 

In [8]:
data Deux a b = Deux a b

instance Bifunctor Deux where
  bimap :: (a -> b) -> (c -> d) -> Deux a c -> Deux b d
  bimap f g (Deux x y) = Deux (f x) (g y)

  first :: (a -> b) -> Deux a c -> Deux b c
  first f (Deux x y) = Deux (f x) y

  second :: (b -> c) -> Deux a b -> Deux a c
  second g (Deux x y) = Deux x (g y)

2. 

In [9]:
data Const a b = Const a

instance Bifunctor Const where
  bimap :: (a -> b) -> (c -> d) -> Const a c -> Const b d
  bimap f _ (Const x) = Const (f x)

  first :: (a -> b) -> Const a c -> Const b d
  first f (Const x) = Const (f x)

  second :: (b -> c) -> Const a b -> Const a c
  second _ (Const x) = Const x

3. 

In [10]:
data Drei a b c = Drei a b c

instance Bifunctor (Drei a) where
  bimap :: (b -> c) -> (d -> e) -> Drei a b d -> Drei a c e
  bimap f g (Drei x y z) = Drei x (f y) (g z)

  first :: (b -> c) -> Drei a b d -> Drei a c d
  first f (Drei x y z) = Drei x (f y) z

  second :: (d -> e) -> Drei a b d -> Drei a b e
  second g (Drei x y z) = Drei x y  (g z)

4. 

In [11]:
data SuperDrei a b c = SuperDrei a b

instance Bifunctor (SuperDrei a) where
  bimap :: (b -> c) -> (d -> e) -> SuperDrei a b d -> SuperDrei a c e
  bimap f _ (SuperDrei x y) = SuperDrei x (f y)

  first :: (b -> c) -> SuperDrei a b d -> SuperDrei a c e
  first f (SuperDrei x y) = SuperDrei x (f y)

  second :: (d -> e) -> SuperDrei a b d -> SuperDrei a b e
  second _ (SuperDrei x y) = SuperDrei x y

5. 

In [12]:
data SemiDrei a b c = SemiDrei a

instance Bifunctor (SemiDrei a) where
  bimap :: (b -> c) -> (d -> e) -> SemiDrei a b d -> SemiDrei a c e
  bimap _ _ (SemiDrei x) = SemiDrei x

  first :: (b -> c) -> SemiDrei a b d -> SemiDrei a c e
  first _ (SemiDrei x) = SemiDrei x

  second :: (d -> e) -> SemiDrei a b d -> SemiDrei a b e
  second _ (SemiDrei x) = SemiDrei x

6. 

In [13]:
data Quadriceps a b c d = Quadzzz a b c d

instance Bifunctor (Quadriceps a b) where
  bimap :: (c -> d) -> (e -> f) -> Quadriceps a b c e -> Quadriceps a b d f
  bimap f g (Quadzzz x y z t) = Quadzzz x y (f z) (g t)

  first :: (c -> d) -> Quadriceps a b c e -> Quadriceps a b d e
  first f (Quadzzz x y z t) = Quadzzz x y (f z) t

  second :: (d -> e) -> Quadriceps a b c d -> Quadriceps a b c e
  second g (Quadzzz x y z t) = Quadzzz x y z (g t)

7. 

In [14]:
data Either' a b = Left' a | Right' b

instance Bifunctor Either' where
  bimap :: (a -> b) -> (c -> d) -> Either' a c -> Either' b d
  bimap f _ (Left' x)  = Left' (f x)
  bimap _ g (Right' y) = Right' (g y)

  first :: (a -> b) -> Either' a c -> Either' b c
  first f (Left' x)  = Left' (f x)
  first _ (Right' y) = Right' y

  second :: (c -> d) -> Either' a c -> Either' a d
  second _ (Left' x)  = Left' x
  second g (Right' y) = Right' (g y)

## 25.7 Monad transformers
Since it is in general impossible to join two unknown monads, we need to reduce the polymorphism and get concrete information about one of the monads that we’re working with to achieve the join.
The other monad remains polymorphic as a variable type argument to the type constructor.

An worse solution would be to implement a `Monad` for every possible combination of types. This can be avoided with monad tranformers, which give the `Monad` for two types, as long as we know what one of the types is.
## 25.8 IdentityT
The `IdentityT` type captures the essence of the monad transformer.
Its bind implementation is:
```haskell
(>>=) :: IdentityT m a -> (a -> IdentityT m b) -> IdentityT m b
(IdentityT ma) >>= f = IdentityT $ ma >>= runIdentityT . f
```
The usage of `runIdentityT` is based on the knowledge of concrete type `IdentityT`.
## 25.9 Finding a pattern
The general patter of a monad transformer is:
```haskell
m (T m b) -> m (m b) -> m b -> T m b
```
where the outer structure `m` is polymorphic and `T` is the concrete type the transformer is for.